In [1]:
!nvidia-smi

Wed May 13 21:37:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 432.00       Driver Version: 432.00       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)   WDDM  | 00000000:01:00.0  On |                  N/A |
| 23%   35C    P8    18W / 250W |    865MiB / 12288MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import os
from os.path import join
import random
import lightgbm as lgb
import time
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, concatenate, Input, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
# from tensorflow.nn import seperable_conv2d
import warnings
 
warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# model_v3 / external feature / atrous conv / spatial pyramid pooling

def atrous_conv(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same", dilation_rate=2)(img_layer)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same", dilation_rate=3)(img_layer)
    conv3 = BatchNormalization()(conv3)
    print('atrous conv3 shape:', conv3.shape)
    pool = AveragePooling2D((2,2))(img_layer)
    print('pool shape:', pool.shape)
    conv4 = Conv2DTranspose(start_neurons, (3, 3), strides=(2, 2), padding="same")(pool)
    print('interpolate shape:', conv4.shape)

    total = concatenate([conv1, conv2, conv3, conv4])
    print('atrous concat:', total.shape)
    total = Conv2D(start_neurons, (1,1), activation='relu', padding='valid')(total)
    print('final atrous shape:', total.shape)
    total = Dropout(0.5)(total)

    return total

def conv_block_2(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
    conv2 = BatchNormalization()(conv2)

    return conv2

def conv_residual(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = conv_block_2(conv1, start_neurons)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def model_v3(input_layer, start_neurons, ext_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.5)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.5)(pool2)

    # 10 x 10 
    convm = atrous_conv(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    print('concat 2:', uconv2.shape)
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.5)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.5)(uconv1)
    
    # process the external feature
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 1 shape:', ext_output.shape)
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 2 shape:', ext_output.shape)
    ext_output = Dropout(0.5)(ext_output)

    output_layer = concatenate([uconv1, ext_output])
    print('final concat shape:', output_layer.shape)

    output_layer = Conv2D(1, (1,1), padding="valid", activation='relu')(output_layer)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 17))
output_layer = model_v3(input_layer, 32, 32)
model = Model(input_layer, output_layer)

input_layer shape: (None, 40, 40, 17)
img_layer shape: (None, 40, 40, 9)
ext_layer shape: (None, 40, 40, 8)
img_layer shape: (None, 40, 40, 9)
conv1 shape: (None, 40, 40, 32)
maxpool 1 shape: (None, 20, 20, 32)
conv2 shape: (None, 20, 20, 64)
maxpool 2 shape: (None, 10, 10, 64)
atrous conv3 shape: (None, 10, 10, 128)
pool shape: (None, 5, 5, 64)
interpolate shape: (None, 10, 10, 128)
atrous concat: (None, 10, 10, 512)
final atrous shape: (None, 10, 10, 128)
convm shape: (None, 10, 10, 128)
concat 2: (None, 20, 20, 128)
upconv2 shape: (None, 20, 20, 64)
upconv1 shape: (None, 40, 40, 32)
ext 1 shape: (None, 40, 40, 32)
ext 2 shape: (None, 40, 40, 32)
final concat shape: (None, 40, 40, 64)
output shape: (None, 40, 40, 1)


In [4]:
# training_set.ftr 파일에 맞는 input 생성 코드
def input_generator_ftr_train():
    # train feather 파일 로드 및 orbit, subset 조합 생성
    TRAIN_FILE = './training_set.ftr'
    train = pd.read_feather(TRAIN_FILE)
    file_cnt = 0
    temp_info = train[['orbit', 'subset']].drop_duplicates()
    orbit = temp_info['orbit'].tolist()
    subset = temp_info['subset'].tolist()
    del temp_info
    size = len(orbit)
  
  # 하나의 이미지 데이터에 해당하는 것만 3차원 변환 및 피처/라벨 생성
    for i in range(size):
        one_img = train.loc[(train['orbit'] == orbit[i]) & (train['subset'] == subset[i])].sort_values('pixel')
        one_img['type1'] = one_img.loc[one_img['type'] == 0, 'type'] + 1
        one_img['type2'] = one_img.loc[(one_img['type'] >= 100) & (one_img['type'] < 200), 'type'] - 100
        one_img['type3'] = one_img.loc[(one_img['type'] >= 200) & (one_img['type'] < 300), 'type'] - 200
        one_img['type4'] = one_img.loc[one_img['type'] >= 300, 'type'] - 300
        one_img.fillna(0, inplace=True)
        one_img['label'] = one_img['precipitation']
        one_img = np.array(one_img.drop(['type', 'orbit', 'subset', 'pixel', 'precipitation'], axis=1)).reshape([40,40,18])
        target = one_img[:,:,-1].reshape(40,40,1)
        cutoff_labels = np.where(target < 0,0, target)
        feature = one_img[:,:,:-1]
        # if (cutoff_labels > 0).sum() < 50: continue
        yield(feature, cutoff_labels)
        file_cnt += 1

train_dataset = tf.data.Dataset.from_generator(input_generator_ftr_train, (tf.float32, tf.float32),
                                               (tf.TensorShape([40,40,17]),tf.TensorShape([40,40,1])))
train_dataset = train_dataset.batch(128).prefetch(1)

In [5]:
# custom loss fuction (maeOverFscore)
def recall_m(y_true, y_pred):
    true_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), dtype=tf.float32)
    possible_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true, 0, 1))), dtype=tf.float32)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), dtype=tf.float32)
    predicted_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_pred, 0, 1))), dtype=tf.float32)
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    idx_one_true = tf.greater_equal(y_true, 0.1)
    y_true = tf.where(idx_one_true == True, 1, 0)
    idx_one_pred = tf.greater_equal(y_pred, 0.1)
    y_pred = tf.where(idx_one_pred == True, 1, 0)
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def mae(y_true, y_pred):
    over_threshold = tf.greater_equal(y_true, 0.1)
    return K.mean(math_ops.abs(y_pred - y_true), axis=-1)

def maeOverFscore(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    return mae(y_true, y_pred) / f1_m(y_true, y_pred)

In [6]:
WEIGHT_DIR = './checkpoint/ver_yyj/'
LOG_DIR = './log/'
logging = TensorBoard(log_dir=LOG_DIR)
checkpoint = ModelCheckpoint(WEIGHT_DIR + 'v03_ep{epoch:02d}-loss{loss:.5f}.ckpt', mode='min',
                             monitor='loss', save_weights_only=True, period=1, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=1, verbose=1)
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=2, verbose=1)
callbacks = [logging, checkpoint, reduce_lr, early_stopping]
adam = tf.keras.optimizers.Adam(learning_rate=1.0e-05)
model.compile(loss=maeOverFscore, optimizer=adam, metrics=[mae, f1_m])

In [7]:
# 이전에 훈련한 weight를 이어서 훈련할 경우 실행
latest = tf.train.latest_checkpoint(WEIGHT_DIR)
print(latest)
model.load_weights(latest)

./checkpoint/ver_yyj/v03_ep01-loss2.20656.ckpt


In [ ]:
start = time.time()
epochs = 5
model_history = model.fit(train_dataset, epochs = epochs, verbose=1,
                          callbacks=callbacks)
print('{}epoch 파일 학습 시간:'.format(epochs), round((time.time()-start)/60, 2))

Epoch 1/5


    139/Unknown - 90s 90s/step - loss: 2.8740 - mae: 0.3011 - f1_m: 0.10 - 146s 73s/step - loss: 2.6775 - mae: 0.3129 - f1_m: 0.117 - 204s 68s/step - loss: 2.6120 - mae: 0.3027 - f1_m: 0.116 - 263s 66s/step - loss: 2.9070 - mae: 0.2970 - f1_m: 0.105 - 322s 64s/step - loss: 3.0231 - mae: 0.2939 - f1_m: 0.100 - 382s 64s/step - loss: 2.8913 - mae: 0.2924 - f1_m: 0.105 - 441s 63s/step - loss: 2.8128 - mae: 0.2895 - f1_m: 0.106 - 501s 63s/step - loss: 2.7539 - mae: 0.2897 - f1_m: 0.109 - 559s 62s/step - loss: 2.7156 - mae: 0.2904 - f1_m: 0.110 - 614s 61s/step - loss: 2.6587 - mae: 0.2901 - f1_m: 0.112 - 668s 61s/step - loss: 2.6141 - mae: 0.2890 - f1_m: 0.114 - 722s 60s/step - loss: 2.5497 - mae: 0.2876 - f1_m: 0.117 - 776s 60s/step - loss: 2.5711 - mae: 0.2901 - f1_m: 0.116 - 830s 59s/step - loss: 2.5908 - mae: 0.2919 - f1_m: 0.116 - 884s 59s/step - loss: 2.6093 - mae: 0.2905 - f1_m: 0.114 - 937s 59s/step - loss: 2.6122 - mae: 0.2902 - f1_m: 0.114 - 991s 58s/step - loss: 2.6245 - mae: 0.28

    275/Unknown - 7616s 54s/step - loss: 1.9521 - mae: 0.3177 - f1_m: 0.17 - 7669s 54s/step - loss: 1.9484 - mae: 0.3179 - f1_m: 0.17 - 7723s 54s/step - loss: 1.9439 - mae: 0.3178 - f1_m: 0.17 - 7777s 54s/step - loss: 1.9413 - mae: 0.3182 - f1_m: 0.17 - 7831s 54s/step - loss: 1.9373 - mae: 0.3185 - f1_m: 0.18 - 7884s 54s/step - loss: 1.9342 - mae: 0.3186 - f1_m: 0.18 - 7937s 54s/step - loss: 1.9320 - mae: 0.3191 - f1_m: 0.18 - 7991s 54s/step - loss: 1.9317 - mae: 0.3190 - f1_m: 0.18 - 8044s 54s/step - loss: 1.9309 - mae: 0.3192 - f1_m: 0.18 - 8098s 54s/step - loss: 1.9322 - mae: 0.3191 - f1_m: 0.18 - 8152s 54s/step - loss: 1.9302 - mae: 0.3192 - f1_m: 0.18 - 8206s 54s/step - loss: 1.9279 - mae: 0.3192 - f1_m: 0.18 - 8259s 54s/step - loss: 1.9251 - mae: 0.3195 - f1_m: 0.18 - 8313s 54s/step - loss: 1.9221 - mae: 0.3202 - f1_m: 0.18 - 8367s 54s/step - loss: 1.9186 - mae: 0.3203 - f1_m: 0.18 - 8421s 54s/step - loss: 1.9158 - mae: 0.3201 - f1_m: 0.18 - 8475s 54s/step - loss: 1.9150 - mae: 0

    410/Unknown - 14917s 54s/step - loss: 1.8740 - mae: 0.3077 - f1_m: 0.176 - 14971s 54s/step - loss: 1.8727 - mae: 0.3079 - f1_m: 0.176 - 15024s 54s/step - loss: 1.8712 - mae: 0.3078 - f1_m: 0.176 - 15078s 54s/step - loss: 1.8693 - mae: 0.3078 - f1_m: 0.177 - 15132s 54s/step - loss: 1.8677 - mae: 0.3079 - f1_m: 0.177 - 15185s 54s/step - loss: 1.8665 - mae: 0.3080 - f1_m: 0.177 - 15239s 54s/step - loss: 1.8644 - mae: 0.3082 - f1_m: 0.177 - 15292s 54s/step - loss: 1.8634 - mae: 0.3084 - f1_m: 0.178 - 15346s 54s/step - loss: 1.8618 - mae: 0.3085 - f1_m: 0.178 - 15399s 54s/step - loss: 1.8598 - mae: 0.3086 - f1_m: 0.178 - 15453s 54s/step - loss: 1.8581 - mae: 0.3087 - f1_m: 0.178 - 15506s 54s/step - loss: 1.8560 - mae: 0.3089 - f1_m: 0.179 - 15560s 54s/step - loss: 1.8550 - mae: 0.3089 - f1_m: 0.179 - 15614s 54s/step - loss: 1.8532 - mae: 0.3090 - f1_m: 0.179 - 15668s 54s/step - loss: 1.8528 - mae: 0.3091 - f1_m: 0.179 - 15722s 54s/step - loss: 1.8527 - mae: 0.3091 - f1_m: 0.179 - 15776s

    541/Unknown - 22157s 54s/step - loss: 1.8020 - mae: 0.3043 - f1_m: 0.180 - 22211s 54s/step - loss: 1.8014 - mae: 0.3042 - f1_m: 0.180 - 22264s 54s/step - loss: 1.8002 - mae: 0.3041 - f1_m: 0.180 - 22318s 54s/step - loss: 1.7989 - mae: 0.3040 - f1_m: 0.180 - 22372s 54s/step - loss: 1.7985 - mae: 0.3040 - f1_m: 0.180 - 22426s 54s/step - loss: 1.7982 - mae: 0.3039 - f1_m: 0.180 - 22479s 54s/step - loss: 1.7989 - mae: 0.3038 - f1_m: 0.180 - 22533s 54s/step - loss: 1.7979 - mae: 0.3037 - f1_m: 0.180 - 22587s 54s/step - loss: 1.7986 - mae: 0.3036 - f1_m: 0.180 - 22640s 54s/step - loss: 1.7984 - mae: 0.3036 - f1_m: 0.180 - 22694s 54s/step - loss: 1.7973 - mae: 0.3035 - f1_m: 0.180 - 22747s 54s/step - loss: 1.7959 - mae: 0.3035 - f1_m: 0.180 - 22801s 54s/step - loss: 1.7947 - mae: 0.3034 - f1_m: 0.180 - 22854s 54s/step - loss: 1.7930 - mae: 0.3034 - f1_m: 0.180 - 22908s 54s/step - loss: 1.7920 - mae: 0.3034 - f1_m: 0.180 - 22962s 54s/step - loss: 1.7907 - mae: 0.3032 - f1_m: 0.180 - 23016s

...

141/541 [======>.......................] - ETA: 9:37:52 - loss: 1.8758 - mae: 0.2355 - f1_m: 0.125 - ETA: 8:53:54 - loss: 1.7895 - mae: 0.2459 - f1_m: 0.138 - ETA: 8:39:40 - loss: 1.7372 - mae: 0.2356 - f1_m: 0.135 - ETA: 8:32:13 - loss: 1.9009 - mae: 0.2301 - f1_m: 0.124 - ETA: 8:27:57 - loss: 1.9684 - mae: 0.2267 - f1_m: 0.118 - ETA: 8:23:38 - loss: 1.8913 - mae: 0.2254 - f1_m: 0.122 - ETA: 8:20:43 - loss: 1.8353 - mae: 0.2224 - f1_m: 0.124 - ETA: 8:18:04 - loss: 1.7966 - mae: 0.2223 - f1_m: 0.127 - ETA: 8:15:56 - loss: 1.7762 - mae: 0.2231 - f1_m: 0.129 - ETA: 8:14:13 - loss: 1.7390 - mae: 0.2232 - f1_m: 0.132 - ETA: 8:12:30 - loss: 1.7112 - mae: 0.2223 - f1_m: 0.133 - ETA: 8:11:02 - loss: 1.6735 - mae: 0.2211 - f1_m: 0.136 - ETA: 8:09:34 - loss: 1.6929 - mae: 0.2234 - f1_m: 0.135 - ETA: 8:08:14 - loss: 1.7149 - mae: 0.2255 - f1_m: 0.135 - ETA: 8:06:48 - loss: 1.7266 - mae: 0.2242 - f1_m: 0.133 - ETA: 8:05:24 - loss: 1.7284 - mae: 0.2239 - f1_m: 0.132 - ETA: 8:04:13 - loss: 1.7322 -

282/541 [==============>...............] - ETA: 6:05:52 - loss: 1.3395 - mae: 0.2560 - f1_m: 0.206 - ETA: 6:04:53 - loss: 1.3384 - mae: 0.2565 - f1_m: 0.206 - ETA: 6:03:53 - loss: 1.3359 - mae: 0.2568 - f1_m: 0.207 - ETA: 6:02:55 - loss: 1.3340 - mae: 0.2570 - f1_m: 0.208 - ETA: 6:01:56 - loss: 1.3328 - mae: 0.2574 - f1_m: 0.208 - ETA: 6:01:00 - loss: 1.3326 - mae: 0.2573 - f1_m: 0.208 - ETA: 6:00:02 - loss: 1.3320 - mae: 0.2575 - f1_m: 0.208 - ETA: 5:59:05 - loss: 1.3335 - mae: 0.2576 - f1_m: 0.208 - ETA: 5:58:07 - loss: 1.3321 - mae: 0.2576 - f1_m: 0.208 - ETA: 5:57:10 - loss: 1.3307 - mae: 0.2576 - f1_m: 0.208 - ETA: 5:56:12 - loss: 1.3291 - mae: 0.2579 - f1_m: 0.209 - ETA: 5:55:12 - loss: 1.3275 - mae: 0.2586 - f1_m: 0.209 - ETA: 5:54:14 - loss: 1.3251 - mae: 0.2588 - f1_m: 0.210 - ETA: 5:53:15 - loss: 1.3231 - mae: 0.2586 - f1_m: 0.210 - ETA: 5:52:16 - loss: 1.3226 - mae: 0.2585 - f1_m: 0.210 - ETA: 5:51:17 - loss: 1.3230 - mae: 0.2586 - f1_m: 0.210 - ETA: 5:50:18 - loss: 1.3225 -

423/541 [======================>.......] - ETA: 3:54:53 - loss: 1.2945 - mae: 0.2491 - f1_m: 0.204 - ETA: 3:53:58 - loss: 1.2935 - mae: 0.2493 - f1_m: 0.205 - ETA: 3:53:04 - loss: 1.2924 - mae: 0.2494 - f1_m: 0.205 - ETA: 3:52:08 - loss: 1.2913 - mae: 0.2495 - f1_m: 0.205 - ETA: 3:51:12 - loss: 1.2902 - mae: 0.2498 - f1_m: 0.206 - ETA: 3:50:16 - loss: 1.2896 - mae: 0.2498 - f1_m: 0.206 - ETA: 3:49:20 - loss: 1.2886 - mae: 0.2498 - f1_m: 0.206 - ETA: 3:48:25 - loss: 1.2889 - mae: 0.2500 - f1_m: 0.206 - ETA: 3:47:29 - loss: 1.2886 - mae: 0.2500 - f1_m: 0.206 - ETA: 3:46:34 - loss: 1.2883 - mae: 0.2500 - f1_m: 0.206 - ETA: 3:45:38 - loss: 1.2881 - mae: 0.2500 - f1_m: 0.206 - ETA: 3:44:42 - loss: 1.2879 - mae: 0.2499 - f1_m: 0.206 - ETA: 3:43:47 - loss: 1.2875 - mae: 0.2499 - f1_m: 0.206 - ETA: 3:42:52 - loss: 1.2868 - mae: 0.2499 - f1_m: 0.206 - ETA: 3:41:57 - loss: 1.2857 - mae: 0.2500 - f1_m: 0.206 - ETA: 3:41:02 - loss: 1.2846 - mae: 0.2501 - f1_m: 0.206 - ETA: 3:40:07 - loss: 1.2836 -

540/541 [============================>.] - ETA: 1:45:53 - loss: 1.2538 - mae: 0.2460 - f1_m: 0.207 - ETA: 1:44:59 - loss: 1.2532 - mae: 0.2460 - f1_m: 0.207 - ETA: 1:44:04 - loss: 1.2524 - mae: 0.2459 - f1_m: 0.207 - ETA: 1:43:10 - loss: 1.2528 - mae: 0.2457 - f1_m: 0.207 - ETA: 1:42:15 - loss: 1.2526 - mae: 0.2458 - f1_m: 0.207 - ETA: 1:41:21 - loss: 1.2526 - mae: 0.2457 - f1_m: 0.207 - ETA: 1:40:26 - loss: 1.2522 - mae: 0.2456 - f1_m: 0.207 - ETA: 1:39:32 - loss: 1.2512 - mae: 0.2456 - f1_m: 0.207 - ETA: 1:38:37 - loss: 1.2516 - mae: 0.2455 - f1_m: 0.207 - ETA: 1:37:43 - loss: 1.2510 - mae: 0.2456 - f1_m: 0.207 - ETA: 1:36:48 - loss: 1.2506 - mae: 0.2456 - f1_m: 0.207 - ETA: 1:35:54 - loss: 1.2500 - mae: 0.2457 - f1_m: 0.207 - ETA: 1:34:59 - loss: 1.2489 - mae: 0.2457 - f1_m: 0.208 - ETA: 1:34:05 - loss: 1.2485 - mae: 0.2457 - f1_m: 0.208 - ETA: 1:33:10 - loss: 1.2480 - mae: 0.2458 - f1_m: 0.208 - ETA: 1:32:16 - loss: 1.2472 - mae: 0.2459 - f1_m: 0.208 - ETA: 1:31:22 - loss: 1.2467 -

111/541 [=====>........................] - ETA: 9:23:56 - loss: 1.5625 - mae: 0.2049 - f1_m: 0.131 - ETA: 8:40:08 - loss: 1.5204 - mae: 0.2183 - f1_m: 0.143 - ETA: 8:25:10 - loss: 1.4669 - mae: 0.2088 - f1_m: 0.142 - ETA: 8:17:26 - loss: 1.5987 - mae: 0.2031 - f1_m: 0.130 - ETA: 8:12:50 - loss: 1.6591 - mae: 0.2006 - f1_m: 0.124 - ETA: 8:09:15 - loss: 1.5951 - mae: 0.1994 - f1_m: 0.128 - ETA: 8:06:29 - loss: 1.5408 - mae: 0.1969 - f1_m: 0.131 - ETA: 8:04:40 - loss: 1.5126 - mae: 0.1973 - f1_m: 0.134 - ETA: 8:03:02 - loss: 1.4952 - mae: 0.1979 - f1_m: 0.136 - ETA: 8:01:36 - loss: 1.4631 - mae: 0.1980 - f1_m: 0.139 - ETA: 8:00:33 - loss: 1.4413 - mae: 0.1971 - f1_m: 0.140 - ETA: 7:59:24 - loss: 1.4107 - mae: 0.1961 - f1_m: 0.143 - ETA: 7:58:26 - loss: 1.4317 - mae: 0.1982 - f1_m: 0.142 - ETA: 7:57:33 - loss: 1.4520 - mae: 0.2002 - f1_m: 0.141 - ETA: 7:56:31 - loss: 1.4607 - mae: 0.1989 - f1_m: 0.139 - ETA: 7:55:03 - loss: 1.4696 - mae: 0.1986 - f1_m: 0.138 - ETA: 7:53:37 - loss: 1.4715 -

# Cross Validation 평가

In [0]:
# cv,ftr 파일에 맞는 input 생성 코드
def input_generator_ftr_cv():
  # cv feather 파일 로드 및 orbit, subset 조합 생성
  CV_FILE = './ftr_data/cv.ftr'
  cv = pd.read_feather(CV_FILE)
  file_cnt = 0
  temp_info = cv[['orbit', 'subset']].drop_duplicates()
  orbit = temp_info['orbit'].tolist()
  subset = temp_info['subset'].tolist()
  del temp_info
  size = len(orbit)
  
  # 하나의 이미지 데이터에 해당하는 것만 3차원 변환 및 피처/라벨 생성
  for i in range(size):
    one_img = cv.loc[(cv['orbit'] == orbit[i]) & (cv['subset'] == subset[i])].sort_values('pixel')
    one_img = np.array(one_img.drop(['orbit', 'subset', 'pixel'], axis=1)).reshape([40,40,15])
    target = one_img[:,:,-1].reshape(40,40,1)
    cutoff_labels = np.where(target < 0,0, target)
    feature = one_img[:,:,:-1]
    # if (cutoff_labels > 0).sum() < 50: continue
    yield(feature, cutoff_labels)
    file_cnt += 1

cv_dataset = tf.data.Dataset.from_generator(input_generator_ftr_cv, (tf.float32, tf.float32),
                                            (tf.TensorShape([40,40,14]),tf.TensorShape([40,40,1])))
cv_dataset = cv_dataset.batch(64)

In [0]:
WEIGHT_DIR = './checkpoint/'
LOG_DIR = './log/'
latest = tf.train.latest_checkpoint(WEIGHT_DIR)
print(latest)
model.load_weights(latest)
logging = TensorBoard(log_dir=LOG_DIR)
result = model.evaluate(cv_dataset)
dict(zip(model.metrics_names, result))

In [0]:
# model_version1
def conv_block_3(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv2)
  conv3 = BatchNormalization()(conv3)

  return conv3

def conv_residual(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = conv_block_3(conv1, start_neurons)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
  conv3 = BatchNormalization()(conv3)

  return conv3

def model_v1(input_layer, start_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = conv_residual(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.25)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 17))
output_layer = model_v1(input_layer, 32)
model = Model(input_layer, output_layer)

In [0]:
# model version 2
def conv_block_3(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv2)
  conv3 = BatchNormalization()(conv3)

  return conv3

def conv_residual(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = conv_block_3(conv1, start_neurons)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
  conv3 = BatchNormalization()(conv3)

  return conv3

def model_v2(input_layer, start_neurons, ext_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = conv_residual(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    print('concat 2:', uconv2.shape)
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.25)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.25)(uconv1)
    
    # process the external feature
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 1 shape:', ext_output.shape)
    # ext_output = Dropout(0.25)(ext_output)
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 2 shape:', ext_output.shape)

    output_layer = concatenate([uconv1, ext_output])
    print('final concat shape:', output_layer.shape)

    output_layer = Conv2D(16, (1,1), padding="valid", activation='relu')(output_layer)
    print('middle output shape:', output_layer.shape)
    output_layer = Conv2D(1, (1,1), padding="valid", activation='relu')(output_layer)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 17))
output_layer = model_v2(input_layer, 32, 64)
model = Model(input_layer, output_layer)

In [0]:
# model_v3 / external feature / atrous conv / 

def atrous_conv(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same", dilation_rate=2)(img_layer)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same", dilation_rate=3)(img_layer)
  conv3 = BatchNormalization()(conv3)
  print('atrous conv3 shape:', conv3.shape)
  pool = AveragePooling2D((2,2))(img_layer)
  print('pool shape:', pool.shape)
  conv4 = Conv2DTranspose(start_neurons, (3, 3), strides=(2, 2), padding="same")(pool)
  print('interpolate shape:', conv4.shape)

  total = concatenate([conv1, conv2, conv3, conv4])
  print('atrous concat:', total.shape)
  total = Conv2D(start_neurons, (1,1), activation='relu', padding='valid')(total)
  print('final atrous shape:', total.shape)
  total = Dropout(0.5)(total)

  return total

def conv_block_2(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
  conv2 = BatchNormalization()(conv2)

  return conv2

def conv_residual(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = conv_block_2(conv1, start_neurons)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
  conv3 = BatchNormalization()(conv3)

  return conv3

def model_v3(input_layer, start_neurons, ext_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.5)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.5)(pool2)

    # 10 x 10 
    convm = atrous_conv(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    print('concat 2:', uconv2.shape)
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.5)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.5)(uconv1)
    
    # process the external feature
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 1 shape:', ext_output.shape)
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 2 shape:', ext_output.shape)
    ext_output = Dropout(0.5)(ext_output)

    output_layer = concatenate([uconv1, ext_output])
    print('final concat shape:', output_layer.shape)

    output_layer = Conv2D(1, (1,1), padding="valid", activation='relu')(output_layer)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 14))
output_layer = model_v3(input_layer, 32, 32)
model = Model(input_layer, output_layer)

input_layer shape: (None, 40, 40, 14)
img_layer shape: (None, 40, 40, 9)
ext_layer shape: (None, 40, 40, 5)
img_layer shape: (None, 40, 40, 9)
conv1 shape: (None, 40, 40, 32)
maxpool 1 shape: (None, 20, 20, 32)
conv2 shape: (None, 20, 20, 64)
maxpool 2 shape: (None, 10, 10, 64)
atrous conv3 shape: (None, 10, 10, 128)
pool shape: (None, 5, 5, 64)
interpolate shape: (None, 10, 10, 128)
atrous concat: (None, 10, 10, 512)
final atrous shape: (None, 10, 10, 128)
convm shape: (None, 10, 10, 128)
concat 2: (None, 20, 20, 128)
upconv2 shape: (None, 20, 20, 64)
upconv1 shape: (None, 40, 40, 32)
ext 1 shape: (None, 40, 40, 32)
ext 2 shape: (None, 40, 40, 32)
final concat shape: (None, 40, 40, 64)
output shape: (None, 40, 40, 1)


In [0]:
# 원본 npy 파일에 맞는 input 생성 코드
def input_generator_npy():
  train_basic_path = 'raw_data'
  train_path_list = sorted(glob.glob(train_basic_path + '/train*'))
  print('디렉토리 개수:', len(train_path_list))
  file_num = 0
  for path in train_path_list:
    files = sorted(glob.glob(path + '/*'))
    print('총 파일 개수:', file_num)
    for file in files:
      data = np.load(file)
      target = data[:, :, -1].reshape(40,40,1)
      cutoff_labels = np.where(target < 0,0, target)
      feature = data[:,:,:-1]
      if (cutoff_labels > 0).sum() < 50: continue
      yield(feature, cutoff_labels)
      file_num += 1

train_dataset = tf.data.Dataset.from_generator(input_generator_npy, (tf.float32, tf.float32),
                                               (tf.TensorShape([40,40,14]),tf.TensorShape([40,40,1])))
train_dataset = train_dataset.batch(8).prefetch(1)

In [0]:
# fusionNet과 external feature 까지 고려하는 모델 (미완성)
def conv_block_3(img_layer, start_neurons):

  img_layer = input_layer[:,:,:,:9]
  ext_layer = input_layer[:,:,:,9:]
  print('input_layer shape:', input_layer.shape)
  print('img_layer shape:', img_layer.shape)
  print('ext_layer shape:', ext_layer.shape)

  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="valid")(img_layer)
  print('conv1 shape:', conv1.shape)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="valid", dilation_rate=2)(img_layer)
  print('conv2 shape:', conv2.shape)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="valid", dilation_rate=3)(img_layer)
  print('conv3 shape:', conv3.shape)
  conv3 = BatchNormalization()(conv3)

  return conv3

input_layer = Input((40, 40, 14))
output_layer = conv_block_3(input_layer, 64)
model = Model(input_layer, output_layer)

input_layer shape: (None, 40, 40, 14)
img_layer shape: (None, 40, 40, 9)
ext_layer shape: (None, 40, 40, 5)
conv1 shape: (None, 38, 38, 64)
conv2 shape: (None, 36, 36, 64)
conv3 shape: (None, 34, 34, 64)
